In [2]:
import numpy as np

In [3]:
def combination(data,gap=5):
    dataset = []
    for idx,value in enumerate(data):
        if(idx<len(data)-gap):
            temp = data[idx:idx+gap]
            temp = temp.reshape(6*gap)
            dataset.append(temp)
    return np.array(dataset)
 
def train_data(gap=5):
    def get_data():#normlize
        data = np.loadtxt('hk.txt')
        x = combination(data,gap)
        y = data[gap:]
        return x,y
    return get_data()

In [4]:
def get_test(gap=5):
    data = np.loadtxt('lotto.txt').astype('int8')
    x = combination(data,gap)
    y = data[gap:]
    return x,y 

In [5]:
x,y = train_data()
test_x,test_y = get_test()
from sklearn import tree
tree_model = tree.DecisionTreeRegressor().fit(x,y)
tree_result = tree_model.predict(test_x).astype('int8')
from sklearn import linear_model
linear_Model = linear_model.LinearRegression().fit(x,y)
linear_result = np.around(linear_Model.predict(test_x)).astype('int8')

In [6]:
del test_x,test_y,x,y

In [7]:
from keras.models import load_model
model = load_model('model.h5')
x,y = train_data(30)
test_x,test_y = get_test(30)
print(test_x.shape)
lstm_result =np.around(model.predict(test_x/50)*50).astype('int8')

Using TensorFlow backend.


(1440, 180)


In [8]:
import random

In [9]:
def test():    
    countRd = 0
    countJoint = 0

    winRd = 0
    winJoint = 0
    for idx in range(len(test_y)):
        print("------開獎對比----"+str(idx))
        rand = np.sort((np.add(np.random.choice(49,6, replace=False),1)))
        print(rand)
        print(linear_result[idx])
        print(tree_result[idx])
        print(lstm_result[idx])
        print(test_y[idx])
        print("-------------------")
        setRd = set(rand)
        setTree = set(tree_result[idx])
        setLinear = set(linear_result[idx]) 
        setLSTM = set(lstm_result[idx])
        setWin = set(test_y[idx])

        prediction = (setLinear&setTree)|(setLinear&setLSTM)|(setLSTM&setTree)
        setJoint =  setLinear|setTree|setLSTM

        if(len(prediction)<=6):
            if(len(prediction)<6): 
                while len(prediction)<6: #Random pick from setJoint until 6 numbers
                    # base on Multi-Model or Decision Tree or  Linear Model or ...
                    for i in random.sample(setLSTM,6-len(prediction)):
                        prediction.add(i)
        else:
            for i in random.sample(prediction,len(prediction)-6):
                prediction.remove(i)
        print(prediction)        
                
        resultRd = setRd&setWin
        resultJoint = prediction&setWin

        if(resultRd!=set()):
            print('Random:',resultRd)
            if(len(resultRd)>=3):
                print('Random 中獎咯')
                winRd += 1
            countRd += 1

        if(resultJoint!=set()):
            print('Joint:',resultJoint)
            if(len(resultJoint)>=3):
                print('Joint 中獎咯')
                winJoint += 1
            countJoint += 1
    return countRd,countJoint,winRd,winJoint

In [13]:
countRds = []
countJoints = []
winRds = []
winJoints =[]

for i in range(1):#batch size 
    countRd,countJoint,winRd,winJoint = test()
    countRds.append(countRd)
    countJoints.append(countJoint)
    winRds.append(winRd)
    winJoints.append(winJoint)
    print('model','hit','win')
    print('Rd',countRd,winRd)
    print('Joint',countJoint,winJoint)

------開獎對比----0
[ 2 13 21 25 30 44]
[ 7 13 21 28 35 42]
[ 5  7 17 18 27 48]
[ 8 15 22 28 36 43]
[13 17 19 26 39 45]
-------------------
{7, 8, 43, 15, 22, 28}
Random: {13}
------開獎對比----1
[ 7 11 18 21 44 46]
[ 6 14 22 29 37 44]
[ 8 18 22 34 48 49]
[ 8 15 22 28 36 43]
[ 3 11 19 20 22 28]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {11}
Joint: {28, 22}
------開獎對比----2
[ 2 18 19 22 30 44]
[ 6 13 19 27 35 42]
[12 19 31 41 42 47]
[ 8 15 22 28 36 43]
[22 25 29 31 34 46]
-------------------
{42, 43, 15, 19, 22, 28}
Random: {22}
Joint: {22}
------開獎對比----3
[ 2 10 13 18 22 24]
[ 8 16 22 30 37 43]
[ 1  2 13 23 35 43]
[ 8 15 22 28 36 43]
[ 1 10 13 33 34 46]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {10, 13}
------開獎對比----4
[12 27 29 30 36 44]
[ 8 15 22 29 36 42]
[18 22 25 32 33 42]
[ 8 15 22 28 36 43]
[ 3  9 13 28 40 48]
-------------------
{36, 22, 8, 42, 43, 15}
------開獎對比----5
[ 6 17 18 21 31 36]
[ 7 14 22 29 36 43]
[ 1  7 22 28 29 49]
[ 8 15 22 28 36 43]
[ 6 27 40 44 45 47]


[13 18 26 28 35 48]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {18}
Joint: {28}
------開獎對比----172
[17 30 32 35 38 41]
[ 7 14 21 30 36 43]
[10 27 33 40 47 48]
[ 8 15 22 28 36 43]
[26 28 29 30 36 37]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {30}
Joint: {36, 28}
------開獎對比----173
[ 7  8 11 20 26 41]
[ 7 14 20 28 36 44]
[22 25 30 37 38 49]
[ 8 15 22 28 36 43]
[ 7 16 32 39 45 49]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {7}
------開獎對比----174
[15 19 20 24 40 47]
[ 7 14 21 29 36 43]
[ 9 17 26 31 35 36]
[ 8 15 22 28 36 43]
[ 3 20 21 22 26 27]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {20}
Joint: {22}
------開獎對比----175
[25 31 39 43 44 46]
[ 9 15 21 28 35 42]
[ 3  5 14 15 39 41]
[ 8 15 22 28 36 43]
[12 22 24 26 27 47]
-------------------
{36, 8, 43, 15, 22, 28}
Joint: {22}
------開獎對比----176
[ 3  5 18 36 45 49]
[ 7 13 20 28 35 43]
[14 26 28 38 39 45]
[ 8 15 22 28 36 43]
[ 1  2  7  8 27 45]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {45}
Joint: {8}

-------------------
{36, 8, 43, 15, 22, 28}
Random: {38}
Joint: {15}
------開獎對比----344
[14 19 20 36 37 48]
[ 7 14 21 28 36 43]
[ 1  3 20 21 35 37]
[ 8 15 22 28 36 43]
[ 1  5 18 25 35 41]
-------------------
{36, 43, 15, 21, 22, 28}
------開獎對比----345
[20 21 26 32 38 41]
[ 8 15 21 28 35 42]
[ 2 11 12 22 24 25]
[ 8 15 22 28 36 43]
[11 22 27 29 35 40]
-------------------
{36, 8, 43, 15, 22, 28}
Joint: {22}
------開獎對比----346
[ 2 15 27 38 39 47]
[ 7 16 24 31 37 43]
[ 2  5 13 37 42 44]
[ 8 15 22 28 36 43]
[ 2  5 12 25 26 38]
-------------------
{36, 37, 43, 15, 22, 28}
Random: {2, 38}
------開獎對比----347
[ 1 19 22 27 34 36]
[ 8 15 21 28 35 41]
[ 5  7 18 26 29 47]
[ 8 15 22 28 36 43]
[ 2 12 31 35 43 47]
-------------------
{36, 8, 43, 15, 22, 28}
Joint: {43}
------開獎對比----348
[ 2  7 21 26 38 42]
[ 8 15 22 30 36 42]
[ 3  6 13 37 40 42]
[ 8 15 22 28 36 43]
[16 24 25 38 42 46]
-------------------
{36, 8, 42, 43, 15, 22}
Random: {42, 38}
Joint: {42}
------開獎對比----349
[ 5 14 17 22 29 32]
[ 6 13 20 28

{36, 8, 43, 15, 22, 28}
Random: {13}
------開獎對比----517
[ 8 14 19 37 40 49]
[ 7 13 21 27 35 43]
[ 6 11 17 31 40 41]
[ 8 15 22 28 36 43]
[ 6 33 36 42 44 45]
-------------------
{36, 8, 43, 15, 22, 28}
Joint: {36}
------開獎對比----518
[ 5  7 13 27 43 47]
[ 6 12 20 27 35 43]
[19 24 34 40 41 46]
[ 8 15 22 28 36 43]
[ 3 12 24 31 43 46]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {43}
Joint: {43}
------開獎對比----519
[11 14 25 27 35 44]
[ 7 14 21 27 35 43]
[ 9 12 25 31 43 44]
[ 8 15 22 28 36 43]
[ 8  9 25 36 40 44]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {25, 44}
Joint: {8, 36}
------開獎對比----520
[ 1  9 10 12 13 28]
[ 7 14 21 28 35 43]
[ 3  7 16 19 44 47]
[ 8 15 22 28 36 43]
[ 2  8 20 23 45 48]
-------------------
{36, 7, 8, 43, 22, 28}
Joint: {8}
------開獎對比----521
[ 3  5  9 15 17 27]
[ 7 14 21 28 35 42]
[ 8 15 19 21 24 44]
[ 8 15 22 28 36 43]
[16 18 21 22 24 38]
-------------------
{36, 8, 15, 21, 22, 28}
Joint: {21, 22}
------開獎對比----522
[ 1  5  9 20 29 42]
[ 7 14 21 28 35 42]


[ 1  6 21 26 33 40]
[ 7 13 20 27 35 42]
[ 9 17 21 22 31 32]
[ 8 15 22 28 36 43]
[ 7  8 24 34 36 38]
-------------------
{36, 8, 43, 15, 22, 28}
Joint: {8, 36}
------開獎對比----691
[11 14 35 39 40 48]
[ 7 13 21 29 36 44]
[ 3  7 28 29 38 48]
[ 8 15 22 28 36 43]
[17 23 33 38 43 48]
-------------------
{36, 7, 8, 15, 28, 29}
Random: {48}
------開獎對比----692
[ 4 10 12 18 28 32]
[ 7 14 22 29 37 44]
[ 2  8 14 25 43 44]
[ 8 15 22 28 36 43]
[21 22 30 37 38 49]
-------------------
{36, 8, 43, 44, 14, 22}
Joint: {22}
------開獎對比----693
[ 6 17 27 37 38 39]
[ 6 13 21 27 35 42]
[ 7  9 22 29 44 49]
[ 8 15 22 28 36 43]
[ 1 17 22 26 29 33]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {17}
Joint: {22}
------開獎對比----694
[ 2 18 21 27 28 31]
[ 8 16 23 30 35 42]
[ 3 18 19 20 37 47]
[ 8 15 22 28 36 43]
[15 16 17 20 32 41]
-------------------
{36, 8, 43, 15, 22, 28}
Joint: {15}
------開獎對比----695
[ 9 11 22 27 37 46]
[ 7 12 20 28 34 43]
[ 3  8 13 17 22 29]
[ 8 15 22 28 36 43]
[ 9 25 28 30 35 41]
--------------

[ 6 14 22 30 31 33]
[ 7 13 20 28 36 44]
[ 1 19 34 39 43 48]
[ 8 15 22 28 36 43]
[17 19 22 25 30 37]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {22, 30}
Joint: {22}
------開獎對比----861
[ 6  9 12 22 27 47]
[ 8 16 22 30 37 42]
[ 1  4  5 13 14 19]
[ 8 15 22 28 36 43]
[11 14 22 34 41 44]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {22}
Joint: {22}
------開獎對比----862
[ 6 11 23 31 39 43]
[ 8 14 22 30 38 44]
[ 8 24 26 30 47 49]
[ 8 15 22 28 36 43]
[ 5 11 21 24 37 39]
-------------------
{36, 8, 15, 22, 28, 30}
Random: {11, 39}
------開獎對比----863
[ 1 11 24 42 45 49]
[ 6 13 20 27 35 42]
[11 13 41 42 43 49]
[ 8 15 22 28 36 43]
[ 5  7  9 22 27 34]
-------------------
{8, 42, 43, 13, 15, 28}
------開獎對比----864
[10 12 35 40 45 48]
[ 7 15 23 29 36 43]
[10 14 29 35 40 46]
[ 8 15 22 28 36 43]
[ 1 24 28 33 43 44]
-------------------
{36, 43, 15, 22, 28, 29}
Joint: {43, 28}
------開獎對比----865
[ 3  5  8 13 30 32]
[ 8 14 22 30 37 44]
[10 17 18 34 48 49]
[ 8 15 22 28 36 43]
[13 14 18 27 29 44]
--

[ 4 13 24 28 39 44]
[ 6 12 20 27 35 44]
[ 5 14 17 33 41 47]
[ 8 15 22 28 36 43]
[10 19 20 24 33 48]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {24}
------開獎對比----1033
[ 2 12 15 24 32 43]
[ 6 13 20 28 36 43]
[18 19 33 43 45 48]
[ 8 15 22 28 36 43]
[ 6 19 20 26 32 47]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {32}
------開獎對比----1034
[ 7 15 26 33 34 46]
[ 8 15 22 29 35 43]
[ 6  8 12 16 32 49]
[ 8 15 22 28 36 43]
[11 12 23 28 40 48]
-------------------
{36, 8, 43, 15, 22, 28}
Joint: {28}
------開獎對比----1035
[ 2  4  8 23 24 25]
[ 8 14 20 28 35 43]
[ 9 15 18 21 24 37]
[ 8 15 22 28 36 43]
[ 3  5  9 12 13 46]
-------------------
{36, 8, 43, 15, 22, 28}
------開獎對比----1036
[ 2  5 27 28 30 44]
[ 7 14 21 29 37 43]
[23 30 32 33 43 44]
[ 8 15 22 28 36 43]
[13 17 18 24 28 32]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {28}
Joint: {28}
------開獎對比----1037
[ 6 11 16 19 24 36]
[ 8 14 22 29 37 44]
[13 16 21 30 36 37]
[ 8 15 22 28 36 43]
[ 8 26 35 37 40 43]
-------------------
{3

[ 8 15 22 28 36 43]
[ 9 13 21 25 38 39]
-------------------
{36, 8, 43, 15, 20, 22}
Random: {39}
------開獎對比----1206
[ 4 10 15 20 29 44]
[ 8 14 22 29 36 43]
[ 9 27 33 38 39 40]
[ 8 15 22 28 36 43]
[ 1  3 25 31 35 44]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {44}
------開獎對比----1207
[21 23 24 36 45 47]
[ 8 14 22 28 36 42]
[15 25 33 38 40 49]
[ 8 15 22 28 36 43]
[11 15 36 41 44 45]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {36, 45}
Joint: {36, 15}
------開獎對比----1208
[ 8 12 25 30 33 34]
[ 7 14 21 29 36 42]
[14 20 21 31 34 42]
[ 8 15 22 28 36 43]
[ 3  8 10 28 38 47]
-------------------
{36, 42, 14, 21, 22, 28}
Random: {8}
Joint: {28}
------開獎對比----1209
[17 25 29 43 46 47]
[ 7 15 22 29 37 44]
[ 3  4 11 23 44 47]
[ 8 15 22 28 36 43]
[ 2  8 22 27 35 43]
-------------------
{36, 43, 44, 15, 22, 28}
Random: {43}
Joint: {43, 22}
------開獎對比----1210
[ 4  8 10 26 41 48]
[ 7 15 23 30 37 42]
[ 4  8 10 35 46 47]
[ 8 15 22 28 36 43]
[24 29 31 32 44 49]
-------------------
{36, 8, 43,

[ 8 15 22 28 36 43]
[ 2  3  4 23 29 42]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {42}
------開獎對比----1378
[ 7 22 27 29 34 37]
[ 7 14 21 29 36 44]
[ 6 13 14 22 30 44]
[ 8 15 22 28 36 43]
[14 21 25 35 36 39]
-------------------
{36, 43, 44, 14, 15, 22}
Joint: {36, 14}
------開獎對比----1379
[ 7 26 27 28 35 40]
[ 6 12 19 26 33 43]
[ 6  7 29 32 42 46]
[ 8 15 22 28 36 43]
[ 5 15 22 23 28 39]
-------------------
{36, 6, 8, 43, 22, 28}
Random: {28}
Joint: {28, 22}
------開獎對比----1380
[ 9 15 16 22 30 45]
[ 7 13 20 28 36 44]
[ 2  3 13 23 34 38]
[ 8 15 22 28 36 43]
[16 17 30 32 35 41]
-------------------
{36, 43, 13, 15, 22, 28}
Random: {16, 30}
------開獎對比----1381
[12 24 29 32 45 49]
[ 7 15 21 28 36 43]
[ 8 15 19 21 24 44]
[ 8 15 22 28 36 43]
[ 1  3 19 31 32 42]
-------------------
{36, 21, 8, 43, 28, 15}
Random: {32}
------開獎對比----1382
[ 4  5 15 25 32 36]
[ 6 14 20 28 36 43]
[21 25 26 33 35 39]
[ 8 15 22 28 36 43]
[14 23 24 27 36 39]
-------------------
{36, 8, 43, 15, 22, 28}
Random: {36}

In [14]:
countRds = np.array(countRds)
countJoints = np.array(countJoints) 
winRds = np.array(winRds)
winJoints = np.array(winJoints) 

In [15]:
print('Rd \tJoint')
print(countRds.mean(),'\t',countJoints.mean())
print(winRds.mean(),'\t',winJoints.mean())

Rd 	Joint
846.0 	 865.0
22.0 	 37.0


Prediction 2018/01/02

In [105]:
oldest = np.loadtxt('lotto.txt').astype('int8')
oldest = oldest[-18:]
print(oldest.shape)

latest = np.array([[4 , 23 , 25 , 28 , 39 , 43],
                   [4 , 8 , 10 , 14 , 41 , 45],
                   [9 , 14 , 17 , 19 , 46 , 47],
                   [2 , 3 , 5 , 28 , 42 , 48],
                   [1 , 12 , 13 , 37 , 41 , 43],
                   [1 , 5 , 8 , 10 , 34 , 35],
                   [16 , 19 , 28 , 37 , 38 , 41],
                   [17 , 19 , 24 , 27 , 31 , 37],
                   [3 , 14 , 18 , 31 , 41 , 42],
                   [12 , 18 , 20 , 21 , 31 , 37],
                   [7 , 8 , 10 , 20 , 24 , 27],
                   [11 , 12 , 15 , 20 , 27 , 47]])
print(latest.shape)
test_new = np.r_[oldest,latest]
print(test_new.shape)
lstm_input = test_new.reshape(1,180)
lstm_output = np.around(model.predict(lstm_input/50)*50).astype('int8')
input_X = test_new[-5:].reshape(1,30)
tree_output = tree_model.predict(input_X).astype('int8')
linear_output = np.around(linear_Model.predict(input_X)).astype('int8')
print(tree_output.shape)
print(lstm_output.shape)

(18, 6)
(12, 6)
(30, 6)
(1, 6)
(1, 6)


In [106]:
print(tree_output.flatten())
print(linear_output.flatten())
print(lstm_output.flatten())
target = [2 , 6 , 12 , 18 , 36 , 46]
print(target)

[ 3 12 19 21 31 48]
[ 8 15 23 30 36 43]
[ 8 15 22 28 36 43]
[2, 6, 12, 18, 36, 46]


In [92]:
predictRandom = set(np.sort((np.add(np.random.choice(49,6, replace=False),1))))
predictTree = set(tree_output.flatten())
predictLinear = set(linear_output.flatten()) 
predictLSTM = set(lstm_output.flatten())
winSet = set(target)
predict = (predictLinear&predictTree)|(predictLinear&predictLSTM)|(predictLSTM&predictTree)
jointSet =  predictLinear|predictTree|predictLSTM
print(jointSet&winSet)
print(predict&winSet)
print(predictRandom&winSet) 

{36, 12}
{36}
set()


Prediction Next Time

In [113]:
latest = np.array([[4 , 23 , 25 , 28 , 39 , 43],
                   [4 , 8 , 10 , 14 , 41 , 45],
                   [9 , 14 , 17 , 19 , 46 , 47],
                   [2 , 3 , 5 , 28 , 42 , 48],
                   [1 , 12 , 13 , 37 , 41 , 43],
                   [1 , 5 , 8 , 10 , 34 , 35],
                   [16 , 19 , 28 , 37 , 38 , 41],
                   [17 , 19 , 24 , 27 , 31 , 37],
                   [3 , 14 , 18 , 31 , 41 , 42],
                   [12 , 18 , 20 , 21 , 31 , 37],
                   [7 , 8 , 10 , 20 , 24 , 27],
                   [11 , 12 , 15 , 20 , 27 , 47],
                   [2 , 6 , 12 , 18 , 36 , 46]])
oldest = np.loadtxt('lotto.txt').astype('int8')
oldest = oldest[-17:]
test_new = np.r_[oldest,latest]
lstm_input = test_new.reshape(1,180)
lstm_output = np.around(model.predict(lstm_input/50)*50).astype('int8')
input_X = test_new[-5:].reshape(1,30)
tree_output = tree_model.predict(input_X).astype('int8')
linear_output = np.around(linear_Model.predict(input_X)).astype('int8')
predictTree = set(tree_output.flatten())
predictLinear = set(linear_output.flatten()) 
predictLSTM = set(lstm_output.flatten())
predict = (predictLinear&predictTree)|(predictLinear&predictLSTM)|(predictLSTM&predictTree)
jointSet =  predictLinear|predictTree|predictLSTM
print(lstm_output.flatten())
print(tree_output.flatten())
print(linear_output.flatten())
print("----- prediction -----------------------------")
print('模型預測開獎號',jointSet)
print('模型推荐開獎號',predict) 

[ 8 15 22 28 36 43]
[ 8 12 30 43 46 48]
[ 7 15 21 29 37 43]
----- prediction -----------------------------
模型預測開獎號 {36, 37, 7, 8, 43, 12, 46, 15, 48, 21, 22, 28, 29, 30}
模型推荐開獎號 {8, 43, 15}
